# SVM
Support vector machine（支持向量机）,是一种分类算法，它的特点在于找出的分类面是最优的。对于二维数据来说，它可以找到一条直线，这是无数条可以分类的直线当中最完美的，因为它恰好在两个类的中间，距离两个类的点都一样远。如果是高维的点，SVM的分界线就是平面或者超平面（hyperplane）。其算法效果可由下图所示：

![svm1](./resources/svm1.jpg)

上图中，Boundary代表决策面，Margin代表分类间隔，Support Vector代表离决策面最近的那些数据点（虚线穿过的点）。

需要注意的是：如果一个类别的数据点跑到了另一类别中，那么它就是该类别的离散值。SVM 的一个特征就是会忽略离散值并找到具有最大边距的超平面。因此，我们可以说，SVM 对于离散值具有鲁棒性。

SVM有核函数，该函数具有将低维数据转化成高维数据的作用，它可将不可分离的问题转换成可分离的问题。

**实际上最优决策面的方向和位置完全取决于选择哪些样本点作为支持向量**。

## 求解最优决策面

### Direct Representation

求解最优决策面最直接的表达（**Direct Representation**）形式如下所示，其中$margin$函数可以输入一个$boundary$，计算正确分类的所有苹果与香蕉到$boundary$的最小距离。

> $argmax_{boundary} \ margin(boundary)$  
> $s.t.$ 所有正确归类的苹果与香蕉到$boundary$的距离都$>=margin$

考虑使用数学表达，定义直线： $y(x) = w^Tx+b$；平面上任意点$x_0$到直线的距离为：$\dfrac{1}{||w||}(w^Tx_0+b)$；对于$N$个训练点中每一个数据记为：$(x_i, y_i)$（点坐标，label{1, -1}）；则上述求解最优决策面的表达可以写成：

> $\arg\max_{w,b}\left\{ \dfrac{1}{||w||}\min_{n}\left[ y_i(w^Tx_i+b) \right] \right\} $  
> $s.t. \ y_i(w^Tx_i+b)\ge margin, \forall i$